#Intro
Penney's game is a 2 player nontransitive game that can be done using a deck of cards or filling some coins. It is needed that each binary state is roughly 50-50

Two players start by guessing a string of binary states such as HHT or TTT  
and the goal is to see who's string appears first. Each string is known to everyone


When first published, Penney's game uses a string of 3 heads or tails. It has also been shown that red-black cards of a deck of cards is approximate to the probabilities

#Goal
This python notebooks seeks 
1. To generalize the game to a string of length n by utilizing markov chains. 
2. To generalize the game to include m players
3. To generalize the game to include multiple states

In [112]:
import numpy as np
import pandas as pd
import itertools as it
import math
from scipy.special import comb

A markov chain can be represented as an adjacency matrix.  
I will be using binary to represent the states where H=0 and T=1  


So 010 = HTH  


The matrix forms a diagonal matrix that resembles some stairsteps

In [2]:
#function to translate decimal to an arbritrary base

def baseb(n, b, length=None):
  #if type(n) is str:
  #  n = int(n)
  #e = n//b
  #q = n%b
  if n == 0:
    return '0'*length
  elif length is None:
    length = math.floor(1+math.log(n,b))

  new_base = ''
  while n!=0:
    r = n%b
    n=n//b
    
    new_base = str(r) + new_base
  lngth = len(new_base)
  if lngth < length:
    new_base = (length-lngth)*'0' + new_base
  return new_base

for i in range(10):
  print(baseb(i, 2, 4),'i =',i)

0000 i = 0
0001 i = 1
0010 i = 2
0011 i = 3
0100 i = 4
0101 i = 5
0110 i = 6
0111 i = 7
1000 i = 8
1001 i = 9


In [163]:
#generating markov matrix

def make_markov(mtx_length, base):

  #get identity/2
  step1 = np.eye(mtx_length//base, dtype=float)/base

  #interweave columns of identity/2 with itself
  step2 = np.zeros((step1.shape[0], step1.shape[0]*base))
  for i in range(base):
    step2[:,i::base] = step1 #interweaving

  #append a copy to the bottom
  step3 = step2.copy()
  for _ in range(base-1):
    step3 = np.append(step3,step2, axis=0)

  #done
  return step3


def get_states(mtx_length, base=2, length = None):
  #turns base to decimal

  #handling bad cases for given length
  min_digits = math.floor(1+math.log(mtx_length-1,base))
  if length is None or length < min_digits:
    length = min_digits

  states = {}
  for i in range(mtx_length):
    states[baseb(i, base, length)] = i
  return states


#length of subset of interest
length = 3
base = 2
mtx_length = base**length #side length of matrix

print(get_states(mtx_length,base,length))

matrix = make_markov(mtx_length, base)

#displaying markov matrix in a dataframe
matrix_df = pd.DataFrame(matrix)

matrix_df

{'000': 0, '001': 1, '010': 2, '011': 3, '100': 4, '101': 5, '110': 6, '111': 7}


,0,1,2,3,4,5,6,7
0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.5,0.5,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5
4,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.5,0.5,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5


A step in the matix has length = base and repeats = base

#Finding the probabilities

The probability is calculated from the markov matrix. Because the game ends when one of the given patterns matches, these states have a 100% of ending on itself again, essentially ending the chain.

So all we need to do is swap the necessary row with the identity matrix, let the matrix converge through multiple multiplication and multiply with a row vector of equal weights

(row vec of prob) = (row vec of uniform weights) * $\lim_{n \to \infty}M^n$  
where M is the markov transition matrix

In [161]:

#function to get the probabilities when comparing 2 strings
def get_probabilities(players, base = 2, length=3):
  '''
  players = list of binary strings like [0b1, 0b111]
  base = numerical base, will be refered to as base
  length = length of string
  '''
  assert all([ length == k for k in [len(i) for i in players] ]), "Given lengths does not all match"

  mtx_length = base**length

  #reference to change base to decimal and vice versa
  states = get_states(mtx_length, base, length) #base to decimal
  base_states = list(states.keys()) #decimal to base

  #representing given string into base and decimal (without duplicates)
  base_players = list(set(players)) 
  decimal_states = list(set([states[i] for i in players])) #decimal representation
  

  #modifying matrix with ending conditions
  #given states ends the markov chain and are represented as a row from the identity matrix
  I = np.eye(mtx_length,mtx_length, dtype = float)
  mtx_n = make_markov(mtx_length, base=base)
  for j in decimal_states: #replacing rows with identity matrix
    mtx_n[j,:] = I[j,:]

  #each state is equally likely
  start = np.ones((1,mtx_length))/mtx_length

  #we want the markov matrix to converge with repeat multiplications
  #100 is close enough when we round the answer
  n = 100
  M = mtx_n
  for _ in range(n):
    M = np.dot(M,M)


  #P = markov matrix
  #winnings = [0.125,...,0.125]*P^n, as n -> infinity
  #row vector of probabilities
  winnings = np.dot(start,M.round(5)).round(5) 

  #storing given strings into a dictionary with probabilities
  winning_dict = dict.fromkeys(base_players,0)
  for i in decimal_states:
    winning_dict[base_states[i]] = winnings[:,i][0]

  return winning_dict

base = 2
length = 3

get_probabilities(['000', '111'], base, length)

{'000': 0.5, '111': 0.5}

Here, the output of get_probabilities comes out with a dictionary with keys as the patterns and values as the probabilities

#Getting the probabilities for each state

This is to see every possible combination. It becomes harder to visualize the comparison with more players as they add a dimension to the array holding it

In [5]:
#this attempt looks through a matrix of v
base = 2
length = 3
mtx_length = base**length

all_probablities = []
states = get_states(mtx_length, base = base)
base_states = list(states.keys())


for row in range(mtx_length):
  temp = []
  for col in range(mtx_length):
    temp.append(get_probabilities([base_states[col], base_states[row]], 
                                  base = base, 
                                  length = length))
  all_probablities.append(temp)

winning_probablities = pd.DataFrame(all_probablities)
winning_probablities

,0,1,2,3,4,5,6,7
0,{'000': 1.0},"{'000': 0.5, '001': 0.5}","{'010': 0.6, '000': 0.4}","{'000': 0.4, '011': 0.6}","{'100': 0.875, '000': 0.125}","{'101': 0.58333, '000': 0.41667}","{'110': 0.7, '000': 0.3}","{'111': 0.5, '000': 0.5}"
1,"{'000': 0.5, '001': 0.5}",{'001': 1.0},"{'010': 0.33333, '001': 0.66667}","{'011': 0.33333, '001': 0.66667}","{'100': 0.75, '001': 0.25}","{'101': 0.375, '001': 0.625}","{'110': 0.5, '001': 0.5}","{'111': 0.3, '001': 0.7}"
2,"{'010': 0.6, '000': 0.4}","{'010': 0.33333, '001': 0.66667}",{'010': 1.0},"{'010': 0.5, '011': 0.5}","{'010': 0.5, '100': 0.5}","{'010': 0.5, '101': 0.5}","{'110': 0.625, '010': 0.375}","{'111': 0.41667, '010': 0.58333}"
3,"{'000': 0.4, '011': 0.6}","{'011': 0.33333, '001': 0.66667}","{'010': 0.5, '011': 0.5}",{'011': 1.0},"{'100': 0.5, '011': 0.5}","{'101': 0.5, '011': 0.5}","{'110': 0.25, '011': 0.75}","{'111': 0.125, '011': 0.875}"
4,"{'100': 0.875, '000': 0.125}","{'100': 0.75, '001': 0.25}","{'010': 0.5, '100': 0.5}","{'100': 0.5, '011': 0.5}",{'100': 1.0},"{'100': 0.5, '101': 0.5}","{'110': 0.66667, '100': 0.33333}","{'111': 0.4, '100': 0.6}"
5,"{'000': 0.41667, '101': 0.58333}","{'101': 0.375, '001': 0.625}","{'010': 0.5, '101': 0.5}","{'101': 0.5, '011': 0.5}","{'100': 0.5, '101': 0.5}",{'101': 1.0},"{'110': 0.66667, '101': 0.33333}","{'111': 0.4, '101': 0.6}"
6,"{'110': 0.7, '000': 0.3}","{'110': 0.5, '001': 0.5}","{'110': 0.625, '010': 0.375}","{'110': 0.25, '011': 0.75}","{'110': 0.66667, '100': 0.33333}","{'110': 0.66667, '101': 0.33333}",{'110': 1.0},"{'111': 0.5, '110': 0.5}"
7,"{'111': 0.5, '000': 0.5}","{'111': 0.3, '001': 0.7}","{'111': 0.41667, '010': 0.58333}","{'111': 0.125, '011': 0.875}","{'111': 0.4, '100': 0.6}","{'111': 0.4, '101': 0.6}","{'111': 0.5, '110': 0.5}",{'111': 1.0}


In [151]:

#listing every possible combination, but this is not sorted

num_players = 2
base=2
length=4
mtx_length = base**length

states = get_states(mtx_length, base=base)
base_states = list(states.keys())

#{state:[[other states, probabilities],...],...}
winning_state = dict.fromkeys(base_states,None)

#printing out every possible combination of probabilities
for j in it.combinations(base_states, num_players): #iterate through combinations
  prob = get_probabilities(j, base=base, length=length)
  print(prob)



{'0001': 0.5, '0000': 0.5}
{'0000': 0.4, '0010': 0.6}
{'0011': 0.6, '0000': 0.4}
{'0000': 0.3, '0100': 0.7}
{'0101': 0.58333, '0000': 0.41667}
{'0000': 0.36364, '0110': 0.63636}
{'0111': 0.63636, '0000': 0.36364}
{'1000': 0.9375, '0000': 0.0625}
{'1001': 0.625, '0000': 0.375}
{'0000': 0.375, '1010': 0.625}
{'0000': 0.375, '1011': 0.625}
{'0000': 0.25, '1100': 0.75}
{'1101': 0.625, '0000': 0.375}
{'1110': 0.68182, '0000': 0.31818}
{'1111': 0.5, '0000': 0.5}
{'0001': 0.66667, '0010': 0.33333}
{'0001': 0.66667, '0011': 0.33333}
{'0001': 0.5, '0100': 0.5}
{'0101': 0.375, '0001': 0.625}
{'0001': 0.57143, '0110': 0.42857}
{'0111': 0.42857, '0001': 0.57143}
{'1000': 0.875, '0001': 0.125}
{'1001': 0.4375, '0001': 0.5625}
{'0001': 0.5625, '1010': 0.4375}
{'0001': 0.5625, '1011': 0.4375}
{'0001': 0.41667, '1100': 0.58333}
{'1101': 0.4375, '0001': 0.5625}
{'1110': 0.5, '0001': 0.5}
{'1111': 0.31818, '0001': 0.68182}
{'0011': 0.5, '0010': 0.5}
{'0010': 0.6, '0100': 0.4}
{'0101': 0.28571, '0010': 0

#Finding the winning pattern

Now that we can compare various patterns, lets try to find the best pattern (given we know the opponent's patterns)

A dominant pattern is one that has probabily over 0.5, making it more likely to win than not. 

This is found by only using combinations that includes the opponent patterns and finding which left over pattern has the biggest probability


The for loop searches through $\text{number of all possible states} \choose \text{number of players}$


In [157]:
#Given list of opponent patterns, find the best pattern
#if there is no dominant pattern, then return next best or say none
def find_best(them, num_players=num_players, base=base, length=length, raw = False):

  states = get_states(base**length, base=base)
  base_states = list(states.keys())

  #if any opponent string length doesnt match picked string length, assert error
  assert all([ length == k for k in [len(i) for i in them] ]), "Opponent lengths does not match given length"

  # searching for possible states that only includes them
  not_them = []
  for j in it.combinations(base_states, num_players):
    them_in_comb = [i in j for i in them]
    if all(them_in_comb):
      prob = get_probabilities(j,base=base,length=length)

      #find states not included in them list and have prob over 0.5
      not_them += [(i,prob[i]) for i in prob if i not in them]
  
  #handling failed searches
  if not_them == []:
    return "None found"
  if raw:
    #return probabilities, no searching
    return not_them

  #finding the maximum probability
  best_prob = 0.0
  for me in not_them:
    if best_prob < me[1]:
      best_prob = me[1]

  #return states that match the best prob
  return [me for me in not_them if me[1] == best_prob]


num_players = 3
base=3
length=3
them = ["012"]

total_combinations = comb(base**length, num_players,exact=True)
print(f"Searching through {total_combinations} possible combinations")
assert total_combinations < 100000, "Too many combinations"

find_best(them=them, num_players=num_players, base=base, length=length, raw=False)


Searching through 2925 possible combinations


AssertionError: ignored

The biggest limitation behind this code is the number of operations needed. Finding the probability requires multiplying a matrix 100 times to approximate the convergence then if we are looking at $\text{number of all possible states} \choose \text{number of players}$*100 operations, this can easily reach over $10^6$ total operations

#Verifying the math
The probabilities above are made theoretically, so it would be nice to see it it can be observed

In [160]:

def list2str(array, base=2):
  '''
  Translate the random walk into a sequence of decimal to encode the state
  Input:
  array = 2d matrix of only 0's and 1's
  '''
  decimals = []

  if len(array.shape) == 1:
    num_flips = array.shape[0]
    num_trials = 1
  else:
    num_flips = array.shape[1]
    num_trials = array.shape[0]

  for trial in range(num_trials):
    walk = []
    for flip in range(3,num_flips+1):
      subwalk = int(''.join(map(str,array[trial,(flip-3):flip])),base)
      #subwalk = '0b'+''.join(map(str,array[trial,(flip-3):flip]))
      walk.append(subwalk)
    decimals.append(walk)

  return np.array(decimals)

def check_str(players, array, base = 2, length=3):
  '''
  Input:
  players=list of binary strings
  array=decimal translating of random walk per subwalk
  string_length=length of string used for the game
  '''
  mtx_length = base**length
  states = get_states(mtx_length, base=base) #binary to decimal
  base_states = list(states.keys()) #decimal to binary
  unique_players = list(set(players)) #removes duplicates

  #make a dictionary from list of strings
  count = dict.fromkeys(unique_players,0)

  for row in range(array.shape[0]):
    for col in range(array.shape[1]):
      sub_state = base_states[array[row,col]] #decimal to binary
      if sub_state in unique_players:
        count[sub_state] += 1
        break
      else:
        assert 'Unknown Error'

  total = np.sum(list(count.values()))

  for i in unique_players:
    count[i] = count[i]/total
  return count


num_flips = 50
num_trials = 10000

base = 3
length = 3

walk = np.random.randint(0,base,(num_trials,num_flips))
compare = ['000','111', '012']
#assert all([ length == k for k in [len(i) for i in compare] ]), "Given lengths does not all match"

print(check_str(compare, 
                array = list2str(walk, base=base), 
                base = base, 
                length=length))

get_probabilities(compare, base=base,length=length)


{'111': 0.3088294448913918, '012': 0.3920957361222848, '000': 0.29907481898632343}


{'000': 0.3, '012': 0.4, '111': 0.3}

Testing out a few comparison seems to agree with theory, nice!